<a href="https://colab.research.google.com/github/joshtburdick/misc/blob/master/plog/Factoring3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Further attempt at factoring

This is to test a simpler variant of the factoring method using loopy belief propagation. But without the loopy belief propagation.

The main idea is, given $n$, to solve $n=(a+b)(a-b) \mod m$, where $m = \prod p_i$ for $P$ (smallish) primes. Then check $\mathrm{GCD}(n, x)$ for $2^{2P}$ numbers $x$ which are derived from $a$ and $b$ using the Chinese Remainder Theorem.

N.B.: I'm not sure it works. Furthermore, it seems fairly impractical, as it requires computing GCD $2^{2P}$ times.

In [37]:
!pip install --quiet modulo

In [38]:
import itertools
import math

import numpy as np

from modulo import modulo

We'll need some Chinese Remainder Theorem utilities.

In [39]:
def solve_mod_primes(x_mod, primes):
    """Given what x is (mod some primes), solve for x.

    x_mod: an array of small integers, such that x % primes[i] == x_mod[i]
    primes: an array of primes

    Returns: x, in the range 1 <= x <= product(primes),
        satisfying x % primes[i] == xmod[i].
    """
    x = modulo(x_mod[0], primes[0])
    for i in range(1, len(primes)):
        x &= modulo(x_mod[i], primes[i])
    return int(x)

Let $n$ be the number to be factored (WLOG assume $n$ is odd). We want to write $n=(a+b)(a-b)$.

**Somewhat dubious** conjecture: WLOG we can assume $a+b=n$ and $a-b=1$.

In [40]:
def factor(n, primes):
    """Factor n using the Chinese Remainder Theorem.

    n: the number to factor
    primes: an array of primes
    Returns: a nontrivial factor of n, or None on failure
    """
    a_mod_m = (n-1) // 2
    b_mod_m = 1
    a = [[a_mod_m % p, -a_mod_m % p] for p in primes]
    b = [[b_mod_m % p, -b_mod_m % p] for p in primes]
    print(f"a = {a}")
    print(f"b = {b}")
    # get all possible a +/- b ("generalized", for however
    # many prime factors)
    a_mod_m = [solve_mod_primes(a1, primes)
        for a1 in itertools.product(*a)]
    b_mod_m = [solve_mod_primes(b1, primes)
        for b1 in itertools.product(*b)]
    # check GCD of each of these
    for (a1,b1) in itertools.product(a_mod_m, b_mod_m):
        f = math.gcd(n, a1+b1)
        if f != 1 and f != n:
            # print(f"f = {f}")
            return f
    return None


Some tests:

In [41]:
primes = [11,13,17]

In [42]:
factor(3*5, primes)

a = [[7, 4], [7, 6], [7, 10]]
b = [[1, 10], [1, 12], [1, 16]]


3

In [43]:
factor(5*7, primes)

a = [[6, 5], [4, 9], [0, 0]]
b = [[1, 10], [1, 12], [1, 16]]


5

In [44]:
primes = [11,13,17,19,23]

In [45]:
factor(29*31, primes)

a = [[9, 2], [7, 6], [7, 10], [12, 7], [12, 11]]
b = [[1, 10], [1, 12], [1, 16], [1, 18], [1, 22]]


29

In [46]:
primes = [11,13,17,19,23,29,31]

In [47]:
factor(37*41, primes)

a = [[10, 1], [4, 9], [10, 7], [17, 2], [22, 1], [4, 25], [14, 17]]
b = [[1, 10], [1, 12], [1, 16], [1, 18], [1, 22], [1, 28], [1, 30]]


37

In [48]:
factor(41*43, primes)

a = [[1, 10], [10, 3], [14, 3], [7, 12], [7, 16], [11, 18], [13, 18]]
b = [[1, 10], [1, 12], [1, 16], [1, 18], [1, 22], [1, 28], [1, 30]]


41

In [49]:
factor(3*47, primes)

a = [[4, 7], [5, 8], [2, 15], [13, 6], [1, 22], [12, 17], [8, 23]]
b = [[1, 10], [1, 12], [1, 16], [1, 18], [1, 22], [1, 28], [1, 30]]


3

In [50]:
factor(47*59, primes)

a = [[0, 0], [8, 5], [9, 8], [18, 1], [6, 17], [23, 6], [22, 9]]
b = [[1, 10], [1, 12], [1, 16], [1, 18], [1, 22], [1, 28], [1, 30]]


59

In [51]:
# just confirming that, even though 47*59 isn't divisible by 11,
# a == 1386 is
a = (47*59-1) // 2
a, a % 11

(1386, 0)

In [52]:
factor(61*67, primes)

a = [[8, 3], [2, 11], [3, 14], [10, 9], [19, 4], [13, 16], [28, 3]]
b = [[1, 10], [1, 12], [1, 16], [1, 18], [1, 22], [1, 28], [1, 30]]


67

## Slightly more testing

It seems to work for small numbers. What about slightly larger numbers?

In [53]:
def is_prime(n):
    if n < 2:
        return False
    for i in range(2, int(math.sqrt(n)) + 1):
        if n % i == 0:
            return False
    return True

def get_primes(n_primes):
    primes = []
    num = 2
    while len(primes) < n_primes:
        if is_prime(num):
            primes.append(num)
        num += 1
    return primes

primes = get_primes(1000)
display(primes[:10]) # display the first 10 primes

[2, 3, 5, 7, 11, 13, 17, 19, 23, 29]

In [54]:
# use some small primes (starting with 7) for m
small_primes = primes[3:11]
print(small_primes)
larger_primes = primes[110:130]
m = math.prod(small_primes)
print(f"m = {m}")

for (a, b) in itertools.combinations(larger_primes, 2):
    n = a*b
    if n > m:
        continue
    print(f"n = {n} = {a} * {b}", flush=True)
    f = factor(n, small_primes)
    if f is not None:
        print(f"f = {f}\n", flush=True)
    else:
        print(f"failed to factor {n} = {a}*{b}", flush=True)
        break

[7, 11, 13, 17, 19, 23, 29, 31]
m = 6685349671
n = 372091 = 607 * 613
a = [[6, 1], [2, 9], [2, 11], [14, 3], [16, 3], [21, 2], [10, 19], [14, 17]]
b = [[1, 6], [1, 10], [1, 12], [1, 16], [1, 18], [1, 22], [1, 28], [1, 30]]
f = 613

n = 374519 = 607 * 617
a = [[2, 5], [6, 5], [7, 6], [4, 13], [14, 5], [16, 7], [6, 23], [19, 12]]
b = [[1, 6], [1, 10], [1, 12], [1, 16], [1, 18], [1, 22], [1, 28], [1, 30]]
f = 607

n = 375733 = 607 * 619
a = [[0, 0], [8, 3], [3, 10], [16, 1], [13, 6], [2, 21], [4, 25], [6, 25]]
b = [[1, 6], [1, 10], [1, 12], [1, 16], [1, 18], [1, 22], [1, 28], [1, 30]]
f = 619

n = 383017 = 607 * 631
a = [[2, 5], [9, 2], [5, 8], [3, 14], [7, 12], [10, 13], [21, 8], [21, 10]]
b = [[1, 6], [1, 10], [1, 12], [1, 16], [1, 18], [1, 22], [1, 28], [1, 30]]
f = 607

n = 389087 = 607 * 641
a = [[6, 1], [8, 3], [11, 2], [12, 5], [2, 17], [9, 14], [11, 18], [18, 13]]
b = [[1, 6], [1, 10], [1, 12], [1, 16], [1, 18], [1, 22], [1, 28], [1, 30]]
f = 641

n = 390301 = 607 * 643
a = [[4, 3

# Questions

- With $a$ and $b$ chosen in this way, will at least one of the "generalized $a+b$" numbers have a nontrivial GCD with $n$? (It does seem to work for some small examples.)

- Given that the number of "generalized $a+b$" numbers grows like $2^{|P|}$, is this practical? (Presumably not.)

- What is this most similar to? (Gemini suggests the quadratic sieve, which seems plausible.)
